### Añadir una columna con identificador a los CSV products, accounts y sales_teams

In [15]:
import os 
import pandas as pd
import numpy as np

#ruta carpeta
folder_path = r"C:\DataFiles\CRM_sales"

#cambio directorio
os.chdir(folder_path) 

#lista nombres archivos csv
csv_file_names = ['accounts.csv',
                  'products.csv',
                  'sales_teams.csv']

for file in csv_file_names:
    df = pd.read_csv(file)
    
    #definir valores para la nueva columna "id"
    max_idx = df.index.max() + 1
    id_values = np.arange(1, max_idx + 1, 1)

    #añadir nueva columna "id" al dataframe 
    df.insert(loc=0, column='id', value=id_values)
    print(f'{file} information: ')
    df.info()

    #guardar dataframe de vuelta al archivo CSV 
    df.to_csv(file, index=False)

accounts.csv information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                85 non-null     int32  
 1   account           85 non-null     object 
 2   sector            85 non-null     object 
 3   year_established  85 non-null     int64  
 4   revenue           85 non-null     float64
 5   employees         85 non-null     int64  
 6   office_location   85 non-null     object 
 7   subsidiary_of     15 non-null     object 
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 5.1+ KB
products.csv information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           7 non-null      int32 
 1   product      7 non-null      object
 2   series       7 non-null      obje

### Crear diccionario con identificador y nombre completo para los tres CSV
Objetivo: tener los datos listos para reemplazar los datos de sales_pipeline.csv

In [16]:
import csv

#nombres de diccionario para cada archivo CSV 
dict_names = ['accounts_dict',
              'products_dict',
              'teams_dict']

for i in range(len(csv_file_names)):
    
    file = csv_file_names[i]
    dict_name = dict_names[i]
    
    with open(file) as csvfile:
        reader = csv.reader(csvfile)
        
        #saltar encabezado 
        next(reader, None)
        
        temporary_dict = {}

        for row in reader:
            #clave: nombre completo; valor: identificador asignado
            temporary_dict[row[1]] = row[0]
        
        #guardar informacion del diccionario temporal al definitivo 
        globals()[dict_name] = temporary_dict

### Verificar si hay valores en el diccionario que no coincidan con los nombres completos en sales_pipeline.csv

In [17]:
sales_df = pd.read_csv('sales_pipeline.csv')

#columna de sales_pipeline y diccionario correspondiente
data = {'product': products_dict,
        'account': accounts_dict,
        'sales_agent': teams_dict}

for column, dict in data.items():
    x = sales_df[~sales_df[column].isin(dict.keys())][column].unique()
    print(f'Missing values for column {column}: {x}')

Missing values for column product: ['GTXPro']
Missing values for column account: [nan]
Missing values for column sales_agent: []


### Modificar valores en el diccionario que no coincidan

In [18]:
#nombre de producto incorrecto en el diccionario: 'GTX Pro' 
products_dict['GTXPro'] = products_dict['GTX Pro']

del products_dict['GTX Pro']

### Reemplazar los valores completos de sales_pipeline por identificadores
Objetivo: optimizar el modelo de datos, posiblemente mejorar la eficiencia de las joins

In [19]:
for column, dict in data.items():
    sales_df[column] = sales_df[column].map(dict)

In [20]:
#vista previa del dataframe
sales_df

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,1C1I7A6R,5,6,9,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,10,2,39,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,10,3,9,Won,2016-10-25,2017-03-07,50.0
3,MV1LWRNH,5,1,11,Won,2016-10-25,2017-03-09,588.0
4,PE84CX4O,33,1,35,Won,2016-10-25,2017-03-02,517.0
...,...,...,...,...,...,...,...,...
8795,9MIWFW5J,3,4,NaN,Prospecting,NaN,NaN,NaN
8796,6SLKZ8FI,3,4,NaN,Prospecting,NaN,NaN,NaN
8797,LIB4KUZJ,3,4,NaN,Prospecting,NaN,NaN,NaN
8798,18IUIUK0,3,4,NaN,Prospecting,NaN,NaN,NaN


In [21]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
dtypes: float64(1), object(7)
memory usage: 550.1+ KB


### Guardar las modificaciones a sales_pipeline.csv

In [22]:
sales_df.to_csv('sales_pipeline.csv', index=False)